In [1]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
# dir_path = './' #local
dir_path = './drive/MyDrive/BAKA/' #colab

In [2]:
#uncomment in colab
from google.colab import drive
drive.mount('/content/drive')
# pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', sep=' ', index_col=False )
# test = np.genfromtxt(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', delimiter=' ',dtype='float64')
df = pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', header=None, sep=' ')
# df.drop(df.index[60:])
# filler = np.zeros((100-len(df), 31))
# df = df.append(pd.DataFrame(filler), ignore_index=True )
# df
if len(df.columns) > 31:
  df = df.drop(columns=[31])
# df = df.drop(df.index[60:])
# df
# sp_df = np.array_split(df, 2)
# sp_df[0]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# for i in ['6']:
#     samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
#     num_tests = int(len(samples)/5);
#     shuffle(samples, random_state = 0)
#     for k in range(0, len(samples)):
# #         df = np.genfromtxt(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], delimiter=' ',dtype='float64')
#         df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
#         df = df.drop(df.index[60:])
#         if len(df.columns) > 31:
#           df = df.drop(columns=[31])
#         if df.isnull().values.any():
#           print(i, " ", samples[k])
#         print(samples[k])
#         display(df)
#         # # if len(df) < 100:
#         # #     filler = np.zeros((100-len(df), 31))
#         # #     df = df.append(pd.DataFrame(filler), ignore_index=True )
#         # x_test.append(df.to_numpy())
#         # y_test.append(int(i));

In [4]:
files = os.listdir(dir_path + 'DataCollection')
files

['0', '2', '3', '4', '1', '5', '6']

In [5]:
x_train = [];
y_train = [];

x_test = [];
y_test = [];
for i in files:
    samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
    num_tests = int(len(samples)/5);
    shuffle(samples, random_state = 0)
    for k in range(0, num_tests):
#         df = np.genfromtxt(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], delimiter=' ',dtype='float64')
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(df.index[60:])
        if len(df.columns) > 31:
          df = df.drop(columns=[31])
        if df.isnull().values.any():
          print(i, " ", samples[k])
        # if len(df) < 100:
        #     filler = np.zeros((100-len(df), 31))
        #     df = df.append(pd.DataFrame(filler), ignore_index=True )
        x_test.append(df.to_numpy())
        y_test.append(int(i));
    
    for k in range(num_tests, len(samples)):
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(df.index[60:])
        if len(df.columns) > 31:
          df = df.drop(columns=[31])
        if df.isnull().values.any():
          print(i, " ", samples[k])
        # df = df.drop(df.index[100:])
        # if len(df) < 100:
        #     filler = np.zeros((100-len(df), 31))
        #     df = df.append(pd.DataFrame(filler), ignore_index=True )
        x_train.append(df.to_numpy())
        y_train.append(int(i));
    
    print(len(samples), ' ', num_tests, ' ', len(samples)- num_tests)
# print(len(x_train))
x_train = np.array(x_train)
y_train = np.array(y_train);

x_test = np.array(x_test)
y_test = np.array(y_test)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)


302   60   242
512   102   410
302   60   242
410   82   328
602   120   482
422   84   338
417   83   334
x_train.shape:  (2376, 60, 31)
y_train.shiape:  (2376,)
x_test.shape:  (591, 60, 31)
y_test.shape:  (591,)


In [6]:
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)



x_train.shape:  (2376, 60, 31)
y_train.shiape:  (2376,)
x_test.shape:  (591, 60, 31)
y_test.shape:  (591,)


In [7]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

In [8]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)

num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, newshape=(-1, num_features))
x_test = min_max_scaler.transform(x_test)
x_test = np.reshape(x_test, newshape=(num_instances, num_time_steps, num_features))

x_test, y_test = shuffle(x_test, y_test, random_state=0)


In [9]:
# x_train = np.delete(x_train, [0,1,2,3,4,5,6,7,8], 0)
# y_train = np.delete(y_train, [0,1,2,3,4,5,6,7,8], 0)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)
# print(y_train)
# print("\n")
# print(y_test)

# y_train = y_train.astype('int')

# print(y_train)


x_train.shape:  (2376, 60, 31)
y_train.shiape:  (2376,)
x_test.shape:  (591, 60, 31)
y_test.shape:  (591,)


In [10]:
model = Sequential()
model.add(LSTM(units=60, input_shape=x_train.shape[1:], return_sequences=True,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(60, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(60, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(60,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# model.add(Dense(30, activation='softmax'))

model.add(Dense(7, activation='softmax',dtype='float64'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 60)            22080     
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 60)            240       
_________________________________________________________________
dropout (Dropout)            (None, 60, 60)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 60)            29040     
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 60)            240       
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 60)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 60)            2

In [11]:
opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-5)

checkpoint_filepath = dir_path + 'Checkpoints/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)


gestures = model.fit(x = x_train,
            y = y_train,
            epochs=50,
            validation_split=0.1, #split 10% of the trainning set for the validation set,
            batch_size=24,
            callbacks=[model_checkpoint_callback],
            shuffle=True
         )

Epoch 1/50
90/90 [==============================] - 17s 109ms/step - loss: 1.8569 - accuracy: 0.4251 - val_loss: 1.7338 - val_accuracy: 0.5966
Epoch 2/50
90/90 [==============================] - 8s 94ms/step - loss: 0.5313 - accuracy: 0.8248 - val_loss: 1.0984 - val_accuracy: 0.8067
Epoch 3/50
90/90 [==============================] - 9s 95ms/step - loss: 0.3717 - accuracy: 0.8846 - val_loss: 0.4516 - val_accuracy: 0.9370
Epoch 4/50
90/90 [==============================] - 8s 93ms/step - loss: 0.3034 - accuracy: 0.9105 - val_loss: 0.2304 - val_accuracy: 0.9328
Epoch 5/50
90/90 [==============================] - 8s 93ms/step - loss: 0.2247 - accuracy: 0.9425 - val_loss: 0.1376 - val_accuracy: 0.9538
Epoch 6/50
90/90 [==============================] - 8s 93ms/step - loss: 0.1945 - accuracy: 0.9522 - val_loss: 0.1196 - val_accuracy: 0.9538
Epoch 7/50
90/90 [==============================] - 8s 94ms/step - loss: 0.2082 - accuracy: 0.9419 - val_loss: 0.1429 - val_accuracy: 0.9454
Epoch 8/50


In [12]:
# model.save(dir_path + 'Models/gestures.h5')

In [13]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=24)
print("test loss, test acc:", results)
model.save(dir_path + 'Models/gestures_dynamic_60', save_format='tf')

Evaluate on test data
25/25 [==============================] - 1s 30ms/step - loss: 0.0464 - accuracy: 0.9882
test loss, test acc: [0.04640038684010506, 0.9881556630134583]


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_dynamic_60/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_dynamic_60/assets


In [14]:
# new_model = load_model(dir_path + 'Models/gestures.h5')
# print("Evaluate on test data")
# results = new_model.evaluate(x_test, y_test, batch_size=80)
# print("test loss, test acc:", results)

In [17]:
min_max_scaler.data_max_

array([179.999, 179.998, 180.   , 179.997, 179.995, 179.997, 179.973,
       180.   , 179.979, 179.938, 562.126, 701.2  , 533.133, 580.836,
       721.897, 545.729, 579.616, 722.18 , 546.661, 579.62 , 727.649,
       503.163, 594.318, 743.124, 508.462, 510.422, 722.803, 501.416,
       173.127, 162.605, 164.575])

In [16]:
model.input

<KerasTensor: shape=(None, 60, 31) dtype=float64 (created by layer 'lstm_input')>